# Plot coarse labels

In [1]:
import pdb
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

import glob
import natsort
from pynwb import NWBHDF5IO
from ndx_events import LabeledEvents, AnnotatedEventsTable, Events

## Count activity and blocklist coarse label durations for each participant

In [6]:
blocklist_labels = True

if blocklist_labels:
    common_acts = ['Blocklist (Data break)', 'Blocklist (Camera move/zoom)', 'Blocklist (Camera occluded)',
                   'Blocklist (Experiment)', 'Blocklist (Private time)',
                   'Blocklist (Tether/bandage)', 'Blocklist (Hands under blanket)', 'Blocklist (Clinical procedure)']
else:
    common_acts = ['Sleep/rest', 'Inactive', 'Talk', 'TV',
                   'Computer/phone', 'Eat', 'Other activity']
    
n_parts = 12
data_lp = '/data2/users/stepeter/files_nwb/downloads/000055/'
vals_all = np.zeros([n_parts, len(common_acts)+1])
for part_ind in range(n_parts):
    fids = natsort.natsorted(glob.glob(data_lp+'sub-'+str(part_ind+1).zfill(2)+'/*.nwb'))
    for fid in fids:
        io = NWBHDF5IO(fid, mode='r', load_namespaces=False)
        nwb = io.read()

        curr_labels = nwb.intervals['epochs'].to_dataframe()
        durations = (curr_labels.loc[:,'stop_time'].values - curr_labels.loc[:,'start_time'].values)

        # Add up durations of each label
        for s, curr_act in enumerate(common_acts):
            for i, curr_label in enumerate(curr_labels['labels'].tolist()):
                if curr_act in curr_label.split(', '):
                    vals_all[part_ind, s] += durations[i]/3600

        # Add up total durations of selected labels (avoid double counting)
        for i, curr_label in enumerate(curr_labels['labels'].tolist()):
            in_lab_grp = False
            for sub_lab in curr_label.split(', '):
                if sub_lab in common_acts:
                    in_lab_grp = True
            vals_all[part_ind, -1] += durations[i]/3600 if in_lab_grp else 0

# Save result
common_acts_col = [val.lstrip('Blocklist (').rstrip(')') for val in common_acts]
df_all = pd.DataFrame(vals_all.round(1), index=['P'+str(val+1).zfill(2) for val in range(n_parts)],
                      columns=common_acts_col+['Total'])
df_all

,Data break,Camera move/zoom,Camera occluded,Experiment,Private time,Tether/bandage,Hands under blanket,Clinical procedure,Total
P01,8.8,0.5,0.0,6.7,2.8,1.3,0.0,0.1,20.3
P02,0.8,1.2,0.0,3.6,7.0,3.7,0.0,0.0,16.2
P03,24.4,0.7,0.0,18.1,1.1,0.6,0.0,0.0,44.9
P04,27.9,1.4,0.0,7.2,2.8,1.7,0.0,0.0,41.1
P05,0.6,0.8,0.3,3.4,2.4,2.0,0.0,0.0,9.5
P06,0.7,0.3,0.0,7.0,1.1,0.8,0.0,0.0,9.9
P07,7.8,1.4,0.2,4.4,2.6,0.6,0.0,0.1,17.0
P08,0.9,0.4,0.2,6.2,2.6,0.6,0.0,0.0,10.8
P09,0.8,1.3,0.0,5.5,1.9,0.9,0.3,0.6,11.2
P10,1.2,1.0,0.0,2.1,3.5,2.9,0.0,0.0,10.7
